In [1]:
import glob
import re
import os
# https://github.com/dimatura/binvox-rw-py/blob/public/binvox_rw.py
import binvox_rw
import torch
from scipy import signal
import numpy as np
import h5py
import json

In [2]:
used_cat_ids = ['02818832', '02876657', '02880940', '03001627', '03046257', '03325088', '03593526', '03636649', '03642806', '03797390', '04379243']
all_binvox = [os.path.normpath(i) for i in glob.glob("model_data/shapes/ShapeNetCore.v2/*/*/models/model_normalized.solid.binvox")]
all_binvox = [i for i in all_binvox if i.split("\\")[3] in used_cat_ids]
print(len(all_binvox))
shape_filename = all_binvox[0]
print(shape_filename)

20318
model_data\shapes\ShapeNetCore.v2\02818832\10c15151ebe3d237240ea0cdca7b391a\models\model_normalized.solid.binvox


In [3]:
with open(shape_filename, 'rb') as f:
    first_model = binvox_rw.read_as_3d_array(f)

In [4]:
# https://github.com/starstorms9/shape/blob/ddbacbd0a9897ac6ca78a42a23732c34c13bda1a/utils.py#L303
x = np.arange(-6, 7, 1)
y = np.arange(-6, 7, 1)
z = np.arange(-6, 7, 1)
xx, yy, zz = np.meshgrid(x,y,z)
gaussian_kernel = np.exp(-(xx**2 + yy**2 + zz**2)/(2*1**2))

def process_voxel(model):
    global gaussian_kernel
    vox = model.data.astype(float)
    
    # Verify the sparsity
    sparsity = vox.mean()
    if sparsity < 0.01:
        return None, sparsity
    elif sparsity < 0.4:
        vox_with_kernel = signal.convolve(vox, gaussian_kernel, mode="same")
        vox = np.where(vox_with_kernel > 0.1, 1, 0)
        # print(sparsity, "new value:", vox.mean())
    
    # Center the voxels
    max_vals_by_axis = [np.max(np.max(vox, axis=2), axis=1), np.max(np.max(vox, axis=2), axis=0), np.max(np.max(vox, axis=1), axis=0)]

    for axis_idx in range(3):
        first_significant_idx = 0
        last_significant_idx = 127
        prev_value = 0
        for idx in range(128):
            if round(max_vals_by_axis[axis_idx][idx]) > 0:
                first_significant_idx = idx
                break
                
        for idx in reversed(range(128)):
            if round(max_vals_by_axis[axis_idx][idx]) > 0:
                last_significant_idx = idx
                break
                
        off_center_val = int((128 - (last_significant_idx - first_significant_idx)) / 2) - first_significant_idx
        vox = np.roll(vox, off_center_val, axis=axis_idx)

    vox_tensor = torch.tensor(vox).float()
    m = torch.nn.AvgPool3d(2, stride=2)
    vox_tensor = torch.round(m(vox_tensor.reshape(1,128,128,128)).reshape(64,64,64)).bool()
    return vox_tensor, sparsity
    
process_voxel(first_model)

(tensor([[[False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False]],
 
         [[False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False]],
 
         [[False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          [False, False, False,  ..., False, False, False],
          ...,
          [False, False, False,  ..., False, Fals

In [5]:
cat_ids = []
model_ids = []
voxel_shapes = []
num_sparsity_under_001 = 0
num_sparsity_under_04 = 0
num_sparsity_over_04 = 0

i = 0
for file_name in all_binvox:
    split_file_name = file_name.split("\\")
    category_id = split_file_name[3]
    model_id = split_file_name[4]
    with open(file_name, 'rb') as f:
        current_model = binvox_rw.read_as_3d_array(f)
        vox_tensor, sparsity = process_voxel(current_model)
        if sparsity < 0.01:
            num_sparsity_under_001 += 1
        elif sparsity < 0.4:
            num_sparsity_under_04 += 1
        else:
            num_sparsity_over_04 += 1
        if vox_tensor is not None:
            cat_ids.append(category_id)
            model_ids.append(model_id)
            voxel_shapes.append(vox_tensor)
    i += 1
    if i % 2500 == 0:
        print(i)

2500
5000
7500
10000
12500
15000
17500
20000


In [6]:
voxel_shapes_numpy = torch.stack(voxel_shapes, dim=0).numpy()
print(voxel_shapes_numpy.shape)

(17934, 64, 64, 64)


In [7]:
with h5py.File('data.hdf5', 'w') as f:
    dset = f.create_dataset("shapes", data=voxel_shapes_numpy)

In [8]:
print(num_sparsity_under_001)
print(num_sparsity_under_04)
print(num_sparsity_over_04)

2384
17631
303


In [9]:
with open('model_metadata.json', 'w') as f:
    json.dump({"cat_ids": cat_ids, "model_ids": model_ids}, f)